<a href="https://colab.research.google.com/github/ravendinata/crep_colab/blob/main/Report_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Tutorial
To run a cell, click on the Play button to the left of each code cell. Usually, you will need to hover on top of the [#] on the left of the code.

NOTE:  
After making changes to any parameter, please run that cell to save the new parameter into the application.

In [ ]:
#@title **Upload Report File**
#@markdown WARNING: Please upload only one file!

from google.colab import files

uploaded = files.upload()
file_names = []
source_file = list(uploaded.keys())[0]

print('File uploaded! Please continue to execute next cell.')

In [ ]:
#@title **Configure Processing Parameters**
#@markdown **What kind of report have you uploaded?**
#@markdown 1. Cohort All Assignment Grades
#@markdown 2. Cohort All Final Grades

mode = 1 # @param ["1", "2"] {type:"raw"}

output_name = "report" # @param {type:"string"}

In [ ]:
# @title **Process Data** (Run this cell to start processing the report)

import os
import sys

import pandas as pd

from pathlib import Path

def process_grouped_data(grouped, output_file_path, value, index, column, sort = False):
  print("[**] Creating output file...")
  with pd.ExcelWriter(output_file_path) as writer:
    for key, item in grouped:
      print(f"[  ] Processing {key}...")
      item = grouped.get_group(key)
      shaped = item.pivot_table(value, index, column, sort = sort)
      shaped.to_excel(writer, sheet_name = key)

  print("[OK] Processing complete!")

def process_file(file_path, output_file_path, mode):
  print("[..] Processing source file...")
  df = pd.read_csv(file_path)

  match mode:
    case 1:
      print("[  ] Grouping data...")
      grouped = df.groupby("course")
      process_grouped_data(grouped, output_file_path, "Grade", ["student name", "class"], "item name")

    case 2:
      print("[  ] Grouping data...")
      grouped = df.groupby("class")
      process_grouped_data(grouped, output_file_path, "Grade", "student name", "course")

    case _:
      raise ValueError("[ER] Invalid report file type.")

# Main

sys.path.append('/drive/content')
if not os.path.exists(source_file):
  raise ValueError(f"[ER] No {source_file} found in current path!")

if output_name[-5:] != ".xlsx":
    output_name += ".xlsx"

print("[OK] Creating file. Please wait...")
process_file(source_file, output_name, mode)
print(f"[OK] File saved to {output_name}")